In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

from collections import OrderedDict
from pathlib import Path
import pandas as pd 
import numpy as np
import re 
import joblib

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

curruser = os.environ.get('USER')

isUseOptWorkspace = False
sys.path.insert(0, './../src')

if isUseOptWorkspace:
    sys.path.insert(0, '/opt/workspace/{}/notebooks/ecom_model/src/'.format(curruser))
    sys.path.insert(0, '/opt/workspace/{}/notebooks/support_library/'.format(curruser)) 
    sys.path.insert(0, '/opt/workspace/{}/libs/python3.5/site-packages/'.format(curruser))
    sys.path.insert(0, '/opt/workspace/{}/notebooks/labdata/lib/'.format(curruser))
else:
    sys.path.insert(0, '/home/{}/notebooks/ecom_model/src/'.format(curruser))
    sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
    sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

from preprosessing import Preproc
from corpora_transform_main import TfIdfFastTextEmbedVectorizer

# from tqdm import tqdm
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

from spark_connector import SparkConnector
from sparkdb_loader import spark
# from spark_helper import SparkHelper
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer

import loader as load

from corpora_process import utils
from processing import Preproc_Response
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import numpy as np
import subprocess
from threading import Thread
from joblib import Parallel, delayed

In [3]:
def drop_col(df, cols: list):
    scol = sdf.columns
    final_cols = [i for i in scol if i not in cols]
    return df.select(*final_cols)

def show(self, n=10):
    return self.limit(n).toPandas()
pyspark.sql.dataframe.DataFrame.show = show

In [4]:
%%time
table_name = 'sendsay_data_predict'
sp = spark(schema='sbx_team_digitcamp', 
           sparkVersion='2.2',
           dynamic_alloc=False,
           numofinstances=40, 
           numofcores=8,
           kerberos_auth=False,
           output=table_name) 
print(sp.sc.version)
hive = sp.sql

2.4.0.cloudera2
CPU times: user 35.9 ms, sys: 19.8 ms, total: 55.7 ms
Wall time: 27.6 s


In [5]:
# sp.exit()

### Collect all user responses for the whole historical period

In [6]:
all_sdf = hive.sql('''
with
--v0.0.7
--04.12.2019 19:20
w_resp as(
      select 
        regexp_extract(organization_id, '(\\\\d{10,12})(?:_)') as inn,
        regexp_extract(organization_id, '(?:_)(\\\\d{3,})')    as kpp,
        organization_id,
        concat_ws('_', organization_id, resp_tracking_cd) as custid_value,
        campaign_nm,
        response_dttm,
        response,
        load_dt
        
       from sbx_team_digitcamp.sme_cdm_v_response
      where lower(response) rlike '^(?=.*sendsay)(?!.*(ошиб|отпис|спам|не удалось|в процессе|доставлено)).*' 
        --and load_dt <= timestamp('%s')
        --and organization_id like '%6446004324%'  
        --and organization_id like '%6454010956%'
        
),
w_send as (
    select 
      regexp_extract(organization_id, '(\\\\d{10,12})(?:_)') as inn,
      regexp_extract(organization_id, '(?:_)(\\\\d{3,})') as kpp,
      organization_id,
      tracking_id as custid_value,
      campaign_nm,
      int_update_dttm as response_dttm,
      'SendSay. Сообщение доставлено' as response,
      load_dt,
      email, email_subject,
      customer_text_01, customer_text_02, customer_text_03, customer_text_04, customer_text_05,
      customer_text_06, customer_text_07, customer_text_08, customer_text_09, customer_text_10,
      customer_text_11, customer_text_12, customer_text_13, customer_text_14, customer_text_15,
      customer_text_16, customer_text_17, customer_text_18, customer_text_19, customer_text_20,
      message_title
    from sbx_team_digitcamp.sme_email_to_sendsay
   where 1=1
     --and load_dt <= timestamp('%s')
     --and organization_id like '%6446004324%'  
     --and organization_id like '%6454010956%'
),
w_only_send as (
    select s.* from (select distinct organization_id from w_resp) r
        left join w_send s on r.organization_id = s.organization_id 
),
w_base as(
    select * from 
    (--text columns in line for response
        select 
         row_number() over (partition by r.organization_id, r.campaign_nm, r.response_dttm, r.response order by r.response_dttm) as rn,
         r.inn, r.kpp, r.organization_id, r.custid_value, r.campaign_nm, r.response_dttm, r.response,
         r.load_dt,
         s.email, s.email_subject,
         s.customer_text_01, s.customer_text_02, s.customer_text_03, s.customer_text_04, s.customer_text_05,
         s.customer_text_06, s.customer_text_07, s.customer_text_08, s.customer_text_09, s.customer_text_10,
         s.customer_text_11, s.customer_text_12, s.customer_text_13, s.customer_text_14, s.customer_text_15,
         s.customer_text_16, s.customer_text_17, s.customer_text_18, s.customer_text_19, s.customer_text_20,
         s.message_title
        from w_resp r
        left join w_only_send s on r.organization_id = s.organization_id
                              and r.campaign_nm = s.campaign_nm                 
         where length(r.inn) >=10
        ) where rn = 1
    
    union

    select '' as rn, * from w_only_send
     where length(inn) >=10
)
select 
       inn
      ,kpp 
      ,campaign_nm 
      ,organization_id
      ,custid_value
      ,response_dttm
      ,response
      ,date_format(response_dttm, 'yyyy-MM-dd') as asday
      ,date_format(last_day(response_dttm), 'yyyy-MM-dd') as last_day
      ,date_format(response_dttm, 'HH:mm:ss') as astime
      ,date_format(response_dttm, 'HH') as ashour
      ,date_format(response_dttm, 'EEEE') as asdayname
      ,load_dt
      ,email
      ,email_subject,
      customer_text_01, customer_text_02, customer_text_03, customer_text_04, customer_text_05,
      customer_text_06, customer_text_07, customer_text_08, customer_text_09, customer_text_10,
      customer_text_11, customer_text_12, customer_text_13, customer_text_14, customer_text_15,
      customer_text_16, customer_text_17, customer_text_18, customer_text_19, customer_text_20,
      message_title    
from w_base   
''') #% (max_load_dt_str, max_load_dt_str))

In [7]:
# all_sdf.cache()

### Collect a list of daytime values based on the distinct values of `last_day` column

In [8]:
last_day_rows = all_sdf.select('last_day').distinct().collect()
last_day_lst = [day['last_day'] for day in last_day_rows if day['last_day'] is not None]
last_day_dt = sorted([datetime.strptime(day_str, '%Y-%m-%d') for day_str in last_day_lst])

In [9]:
last_day_dt[-1], last_day_dt[-2], last_day_dt[-3], last_day_dt[-4]

(datetime.datetime(2020, 6, 30, 0, 0),
 datetime.datetime(2020, 5, 31, 0, 0),
 datetime.datetime(2020, 4, 30, 0, 0),
 datetime.datetime(2020, 3, 31, 0, 0))

### SAMPLING FOR VALIDATION DATASET 

In [10]:
val_sdf = all_sdf.filter((f.col('response_dttm') >= last_day_dt[-4]))
# val_sdf.cache()

In [11]:
# val_sdf.count()

## Find min value of `response_dttm` in VAL dataset

In [12]:
resp_dttm_min = val_sdf.select(f.min('response_dttm')).collect()
resp_dttm_min_str = resp_dttm_min[0]['min(response_dttm)']
resp_dttm_min_str

datetime.datetime(2020, 3, 31, 0, 0, 14)

## Roll back by 3 month from the min value of `last_open_time` and generate a temp dataframe for feature generation process

In [13]:
sendsay_delivered = 'SendSay. Сообщение доставлено'
sendsay_openpage  = 'SendSay. Открытие страницы'
sendsay_openlink  = 'SendSay. Переход по ссылке' 

In [14]:
val_sdf = val_sdf \
        .withColumn('first_send_time', 
                    f.min(f.when(f.col('response') == sendsay_delivered, f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('last_send_time', 
                    f.max(f.when(f.col('response') == sendsay_delivered, f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('first_open_time', 
                    f.min(f.when((f.col('response') == sendsay_openpage), f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('last_open_time', 
                    f.max(f.when((f.col('response') == sendsay_openpage), f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('first_click_time', 
                    f.min(f.when((f.col('response') == sendsay_openlink), f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('last_click_time', 
                    f.max(f.when((f.col('response') == sendsay_openlink),f.col('response_dttm')).otherwise(None))\
                                      .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))

In [15]:
shiftd = 90
shift_back  = val_sdf.select(f.last_day(f.add_months(f.trunc(f.from_unixtime(f.unix_timestamp(f.min('last_open_time')) - shiftd*24*60*60),'MM'),-1)).alias('monthback')).collect()
shift_back  = shift_back[0]['monthback']
shift_back

datetime.date(2020, 1, 31)

In [16]:
sdf = all_sdf.filter((f.col('response_dttm') >= shift_back))
# sdf.cache()

In [17]:
# sdf.count()

In [18]:
del val_sdf

# Main Features Generation Pipeline

## Find inactive users (users without any open/click activity among overall history of communication) 

In [19]:
sdf = sdf \
        .withColumn('first_send_time', 
                    f.min(f.when(f.col('response') == sendsay_delivered, f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('last_send_time', 
                    f.max(f.when(f.col('response') == sendsay_delivered, f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('first_open_time', 
                    f.min(f.when((f.col('response') == sendsay_openpage), f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('last_open_time', 
                    f.max(f.when((f.col('response') == sendsay_openpage), f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('first_click_time', 
                    f.min(f.when((f.col('response') == sendsay_openlink), f.col('response_dttm')).otherwise(None))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))\
        .withColumn('last_click_time', 
                    f.max(f.when((f.col('response') == sendsay_openlink),f.col('response_dttm')).otherwise(None))\
                                      .over(Window.partitionBy('inn', 'kpp', 'campaign_nm')))

In [20]:
inactive_freshhold_min = 2
inactive_freshhold_max = 20

In [21]:
sdf = sdf.withColumn('sum_open_click', 
                     f.sum(f.when((f.col('response') == sendsay_openpage)|\
                                  (f.col('response') == sendsay_openlink),1).otherwise(0))\
                                                           .over(Window.partitionBy('inn', 'kpp', 'email')))

sdf = sdf.withColumn('sum_open_click_camp', 
                     f.sum(f.when((f.col('response') == sendsay_openpage)|\
                                  (f.col('response') == sendsay_openlink),1).otherwise(0))\
                                                           .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))

sdf = sdf.withColumn('delivery_all_cnt', 
                                         f.sum((f.col('response') == sendsay_delivered).cast('integer')) \
                                          .over(Window.partitionBy('inn', 'kpp', 'email')).cast('integer'))

In [22]:
sdf = sdf.withColumn('fresh_user', f.when((f.col('sum_open_click') == 0) & 
                                          (f.col('delivery_all_cnt') <= inactive_freshhold_min), 1)\
                                    .when((f.col('sum_open_click') == 0) & 
                                          (f.col('delivery_all_cnt') > inactive_freshhold_min), 2).otherwise(0))

In [23]:
sdf = sdf.drop(*['delivery_all_cnt','sum_open_click'])

### Retrospective Features Generation Pipeline

- total biased time events (non-cumulated)

In [24]:
daysback = 30

for i in range(1,4):
    
    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_send_time')) - daysback*(i)*24*60*60, 
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_send_time')) - daysback*(i-1)*24*60*60, 
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())    
    
    sdf = sdf \
            .withColumn('first_send_time_m{}1'.format(str(i)), 
                        f.min(f.when((f.col('response') == sendsay_delivered) &\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), f.col('response_dttm'))\
                                    .otherwise(None))\
                        .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))\
            .withColumn('last_send_time_m{}1'.format(str(i)), 
                        f.max(f.when((f.col('response') == sendsay_delivered) &\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), f.col('response_dttm'))\
                                    .otherwise(None))\
                        .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))

for i in range(1,4):            

    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i)*24*60*60, 
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i-1)*24*60*60, 
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    sdf = sdf \
            .withColumn('first_open_time_m{}1'.format(str(i)), 
                        f.min(f.when((f.col('response') == sendsay_openpage) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), f.col('response_dttm'))\
                                    .otherwise(None))\
                        .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))\
            .withColumn('last_open_time_m{}1'.format(str(i)), 
                        f.max(f.when((f.col('response') == sendsay_openpage) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), f.col('response_dttm'))\
                                    .otherwise(None))\
                        .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))
            
for i in range(1,4):
    
    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_click_time')) - daysback*(i)*24*60*60,
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_click_time')) - daysback*(i-1)*24*60*60,
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())
    
    sdf = sdf \
            .withColumn('first_click_time_m{}1'.format(str(i)), 
                        f.min(f.when((f.col('response') == sendsay_openlink) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), f.col('response_dttm'))\
                                    .otherwise(None))\
                        .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))\
            .withColumn('last_click_time_m{}1'.format(str(i)), 
                        f.max(f.when((f.col('response') == sendsay_openlink) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), f.col('response_dttm'))\
                                    .otherwise(None))\
                        .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))

## Calculate monthly biased time events with `status == sendsay_openpage`

### All statistics

- ACTIVE USERS: total biased time events (non-cumulated)

In [25]:
for i in range(1,4):
    
    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i)*24*60*60, 
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i-1)*24*60*60, 
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())     
    sdf = sdf \
            .withColumn('last_open_all_cnt_m{}1'.format(str(i)), 
                        f.sum(f.when((f.col('response') == sendsay_openpage) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), 1)\
                                                       .otherwise(0))\
                                                 .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')).cast('integer'))

### Daytime statistics

- biased time events per day (non-cumulated)

In [26]:
for i in range(1,4):
    
    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i)*24*60*60, 
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i-1)*24*60*60, 
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())     
    sdf = sdf \
            .withColumn('last_open_day_cnt_m{}1'.format(str(i)), 
                        f.sum(f.when((f.col('response') == sendsay_openpage) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), 1)\
                                                       .otherwise(0))\
                                                 .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 
                                                                          'email', 'asdayname')).cast('integer'))

## Calculate monthly biased time events with `status == sendsay_openlink`

### All statistics

- total biased time events (non-cumulated)

In [27]:
for i in range(1,4):
    
    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_click_time')) - daysback*(i)*24*60*60, 
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_click_time')) - daysback*(i-1)*24*60*60, 
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())    
    
    sdf = sdf \
            .withColumn('last_click_all_cnt_m{}1'.format(str(i)), 
                        f.sum(f.when((f.col('response') == sendsay_openlink) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), 1)\
                                                       .otherwise(0))\
                                                 .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')).cast('integer'))

### Daytime statistics

- biased time events per day (non-cumulated)

In [28]:
for i in range(1,4):
    
    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_click_time')) - daysback*(i)*24*60*60, 
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_click_time')) - daysback*(i-1)*24*60*60, 
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())  
    
    sdf = sdf \
            .withColumn('last_click_day_cnt_m{}1'.format(str(i)), 
                        f.sum(f.when((f.col('response') == sendsay_openlink) &\
                                     (f.col('fresh_user')==0)&\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), 1)\
                                                       .otherwise(0))\
                                                 .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 
                                                                          'email', 'asdayname')).cast('integer'))

## Calculate time differences between events with `status == sendsay_openpage`

In [29]:
for i in range(1,4):
    sdf = sdf \
            .withColumn('diff_lo_fo_m{}1'.format(str(i)), ((f.unix_timestamp('last_open_time_m{}1'.format(str(i))) -
                                                                   f.unix_timestamp('first_open_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_lo_fs_m{}1'.format(str(i)), ((f.unix_timestamp('last_open_time_m{}1'.format(str(i))) -
                                                                   f.unix_timestamp('first_send_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_lo_ls_m{}1'.format(str(i)), ((f.unix_timestamp('last_open_time_m{}1'.format(str(i))) -
                                                                   f.unix_timestamp('last_send_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_fo_fs_m{}1'.format(str(i)), ((f.unix_timestamp('first_open_time_m{}1'.format(str(i))) -
                                                                   f.unix_timestamp('first_send_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_fo_ls_m{}1'.format(str(i)), ((f.unix_timestamp('first_open_time_m{}1'.format(str(i))) -
                                                                   f.unix_timestamp('last_send_time_m{}1'.format(str(i))))/60)
                       )   

## Calculate time differences between events with `status == sendsay_openlink`

In [30]:
for i in range(1,4):
    sdf = sdf \
            .withColumn('diff_lcl_fcl_m{}1'.format(str(i)), ((f.unix_timestamp('last_click_time_m{}1'.format(str(i))) -
                                                                     f.unix_timestamp('first_click_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_lcl_fs_m{}1'.format(str(i)), ((f.unix_timestamp('last_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('first_send_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_lcl_ls_m{}1'.format(str(i)), ((f.unix_timestamp('last_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('last_send_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_lcl_fo_m{}1'.format(str(i)), ((f.unix_timestamp('last_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('first_open_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_lcl_lo_m{}1'.format(str(i)), ((f.unix_timestamp('last_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('last_open_time_m{}1'.format(str(i))))/60)
                       )   


    sdf = sdf \
            .withColumn('diff_fcl_fs_m{}1'.format(str(i)), ((f.unix_timestamp('first_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('first_send_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_fcl_ls_m{}1'.format(str(i)), ((f.unix_timestamp('first_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('last_send_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_fcl_fo_m{}1'.format(str(i)), ((f.unix_timestamp('first_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('first_open_time_m{}1'.format(str(i))))/60)
                       )\
            .withColumn('diff_fcl_lo_m{}1'.format(str(i)), ((f.unix_timestamp('first_click_time_m{}1'.format(str(i))) -
                                                                    f.unix_timestamp('last_open_time_m{}1'.format(str(i))))/60)
                       )     

## Calculate events regarding to `status == sendsay_delivered`

- biased delivery count total (non-cumulated)

In [31]:
for i in range(1,4):
    
    dttm_left = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i)*24*60*60, 
                                'yyyy-MM-dd HH:mm:ss').cast(TimestampType())

    dttm_right = f.from_unixtime(f.unix_timestamp(f.col('last_open_time')) - daysback*(i-1)*24*60*60, 
                                 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())     
    
    sdf = sdf \
            .withColumn('delivery_all_cnt_m{}1'.format(str(i)), 
                        f.sum(f.when((f.col('response') == sendsay_delivered) &\
                                     ((f.col('response_dttm')> dttm_left) &\
                                      (f.col('response_dttm')<=dttm_right)), 1)\
                                                       .otherwise(0))\
                                                 .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')).cast('integer'))

## Calculate overall/monthly `open_day_rate/click_day_rate`

In [32]:
def get_delivery_rate(triggered_count: int, delivery_count: int):
    """
    Get the rate of success messages.
    """
    if delivery_count == 0:
        return 0.0
    
    delivery_rate = (triggered_count / delivery_count) * 100  # rate of success messages
    return delivery_rate


get_delivery_rate_udf = f.udf(get_delivery_rate, FloatType())

- biased open/click rate total (non-cumulated)

In [33]:
for i in [str(ii) for ii in range(1,4)]:
#     sdf = sdf.withColumn('open_day_rate_m'+i, get_delivery_rate_udf("last_open_time_sum_m"+i, "deliverycnt"))
    sdf = sdf.withColumn('open_on_dlv_all_rate_m'+i+'1', 
                         get_delivery_rate_udf("last_open_all_cnt_m"+i+'1', "delivery_all_cnt_m"+i+'1'))
#     sdf = sdf.withColumn('open_on_dlv_all_dist_rate_m'+i+'1', 
#                          get_delivery_rate_udf("last_open_all_cnt_dist_m"+i+'1', "delivery_all_cnt_m"+i+'1'))
#     sdf = sdf.withColumn('click_day_rate_m'+i, get_delivery_rate_udf("last_click_time_sum_m"+i, "deliverycnt"))
    sdf = sdf.withColumn('clk_on_dlv_all_rate_m'+i+'1', 
                         get_delivery_rate_udf("last_click_all_cnt_m"+i+'1', "delivery_all_cnt_m"+i+'1'))
#     sdf = sdf.withColumn('clk_on_dlv_all_dist_rate_m'+i+'1', 
#                          get_delivery_rate_udf("last_click_all_cnt_dist_m"+i+'1', "delivery_all_cnt_m"+i+'1'))    
    sdf = sdf.withColumn('clk_on_open_all_rate_m'+i+'1', 
                         get_delivery_rate_udf("last_click_all_cnt_m"+i+'1', "last_open_all_cnt_m"+i+'1'))
#     sdf = sdf.withColumn('clk_on_open_all_dist_rate_m'+i+'1', 
#                          get_delivery_rate_udf("last_click_all_cnt_dist_m"+i+'1', "last_open_all_cnt_dist_m"+i+'1'))     

- biased open/click rate per day (non-cumulated)

In [34]:
for i in [str(ii) for ii in range(1,4)]:
#     sdf = sdf.withColumn('open_day_rate_m'+i, get_delivery_rate_udf("last_open_time_sum_m"+i, "deliverycnt"))
    sdf = sdf.withColumn('open_on_dlv_day_rate_m'+i+'1', 
                         get_delivery_rate_udf("last_open_day_cnt_m"+i+'1', "delivery_all_cnt_m"+i+'1'))
#     sdf = sdf.withColumn('open_on_dlv_day_dist_rate_m'+i+'1', 
#                          get_delivery_rate_udf("last_open_day_cnt_dist_m"+i+'1', "delivery_all_cnt_m"+i+'1'))    
#     sdf = sdf.withColumn('click_day_rate_m'+i, get_delivery_rate_udf("last_click_time_sum_m"+i, "deliverycnt"))
    sdf = sdf.withColumn('clk_on_dlv_day_rate_m'+i+'1', 
                         get_delivery_rate_udf("last_click_day_cnt_m"+i+'1', "delivery_all_cnt_m"+i+'1'))
#     sdf = sdf.withColumn('clk_on_dlv_day_dist_rate_m'+i+'1', 
#                          get_delivery_rate_udf("last_click_day_cnt_dist_m"+i+'1', "delivery_all_cnt_m"+i+'1')) 
    sdf = sdf.withColumn('clk_on_open_day_rate_m'+i+'1', 
                         get_delivery_rate_udf("last_click_day_cnt_m"+i+'1', "last_open_day_cnt_m"+i+'1'))
#     sdf = sdf.withColumn('clk_on_open_day_dist_rate_m'+i+'1', 
#                          get_delivery_rate_udf("last_click_day_cnt_dist_m"+i+'1', "last_open_day_cnt_dist_m"+i+'1'))    

In [35]:
import locale
from functools import cmp_to_key

customer_attr = ['customer_text_0{}'.format(str(i)) if i<10 
                 else 'customer_text_{}'.format(str(i)) for i in range(1,21)]

first_order_cols = \
[
    'inn', 
    'kpp',
    'campaign_nm', 
    'email',
    'organization_id',
    'custid_value',
    'response',
    'response_dttm',
    'asday',
    'asdayname',
    'ashour',
    'astime',     
    'email_subject',
    'message_title',]\
+\
customer_attr

reordered_cols = \
first_order_cols + sorted(set(sdf.columns) -
                          set(first_order_cols) -
                          set(['load_dt','last_day']), key=cmp_to_key(locale.strcoll)) + ['load_dt','last_day']

In [36]:
sdf = sdf.select(*reordered_cols)

## Cut all daytime events that aren't related to the Test datased (cut records in window of 3month back from min responce date) 

In [37]:
sdf = sdf.filter((f.col('response_dttm') >= resp_dttm_min_str)) 

In [38]:
# sdf.count()

In [39]:
# sdf = sdf.withColumnRenamed('inn','inn_')
# sdf = sdf.withColumnRenamed('kpp','kpp_')
# sdf = sdf.withColumnRenamed('campaign_nm','campaign_nm_')
# sdf = sdf.withColumnRenamed('email','email_')

In [40]:
# cols = reordered_cols

# cond = [val_sdf.inn         == sdf.inn_, 
#         val_sdf.kpp         == sdf.kpp_,
#         val_sdf.campaign_nm == sdf.campaign_nm_,
#         val_sdf.email       == sdf.email_,
#         #train_sdf.response    == target_sdf.response_,
#         #train_sdf.response_dttm == target_sdf.response_dttm_
#        ]

# fin_sdf = val_sdf.join(sdf, on=cond, how='left_outer')
# fin_sdf = fin_sdf.select(*cols)

In [41]:
# data condition
# spdf = hive.sql("select * from T_TEAM_SPEECH_ANALYTICS.RESP_INN_KPP_CAMP")
# rows = spdf.select('load_dt').distinct().collect()

# prev_load_dt = sorted([row['load_dt'] for row in rows])[-2]
# prev_load_dt_str = datetime.strftime(prev_load_dt, format='%Y-%m-%d')
# prev_load_dt_str

In [42]:
# sdf.groupby(['fresh_user']).agg({'fresh_user':'count'}).show()

## Remove events with only delivery status among users with para `fresh_user` = 0

In [38]:
sdf = sdf.filter("(response != 'SendSay. Сообщение доставлено' and fresh_user = 0) or (sum_open_click_camp = 0 and fresh_user = 0) or (fresh_user != 0)")

## Register temp table

In [43]:
sdf.registerTempTable("tmp_resp_inn_kpp_camp")
hive.cacheTable('tmp_resp_inn_kpp_camp')

## Generate features required for best day/time calculation

In [44]:
best_time_sdf = sdf \
    .withColumn('day_open_cnt', 
                    f.sum(f.when(f.col('response') == sendsay_openpage, 1).otherwise(0))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email', 'asdayname')).cast('integer'))\
    .withColumn('day_hour_open_cnt', 
                    f.sum(f.when(f.col('response') == sendsay_openpage, 1).otherwise(0))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email', 'asdayname', 'ashour')).cast('integer'))\
    .select(*['inn', 'kpp', 'campaign_nm', 'email', 'response', 'asdayname', 'ashour', 'day_open_cnt', 'day_hour_open_cnt'])

In [45]:
best_time_sdf = best_time_sdf.filter(''' response != "SendSay. Сообщение доставлено" ''')\
         .withColumn('top_day', f.dense_rank().over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email', 'asdayname')\
                                              .orderBy(f.col('day_open_cnt').desc())))\
                                              .filter('''top_day = 1''')    
    
best_time_sdf = best_time_sdf.\
        withColumn('top_hour', f.dense_rank().over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email', 'asdayname')\
                                             .orderBy(f.col('day_hour_open_cnt').desc()))).filter('''top_hour = 1''')
    
best_time_sdf = best_time_sdf.drop(*['top_day', 'top_hour', 'day_hour_open_cnt', 'response'])

### Groupby up to `email` (keep on one first record per one email)

In [46]:
best_time_sdf = best_time_sdf.groupBy('inn', 'kpp', 'campaign_nm', 'email').agg(f.first('asdayname').alias('asdayname'), 
                                                                                f.first('ashour').alias('ashour'),
                                                                                f.first('day_open_cnt').alias('day_open_cnt'))

In [47]:
best_time_sdf.cache()

DataFrame[inn: string, kpp: string, campaign_nm: string, email: string, asdayname: string, ashour: string, day_open_cnt: int]

In [48]:
best_time_sdf.count()

2725213

In [49]:
best_time_sdf.show()

,inn,kpp,campaign_nm,email,asdayname,ashour,day_open_cnt
0,010407628324,,ЦА-БИЗНДИР34ПОС-0420,tehnomir01@mail.ru,Tuesday,15,1
1,0105001202,010501001,E-MAIL_SERVICE_CAMP,9284666801@mail.ru,Monday,08,1
2,0105010398,010501001,E-MAIL_SERVICE_CAMP,arina0605@gmail.com,Saturday,18,3
3,0105012701,010501001,E-MAIL_SERVICE_CAMP,tdmnormal@mail.ru,Saturday,01,0
4,0105021640,010501001,E-MAIL_SERVICE_CAMP,miriadakkm@mail.ru,Sunday,18,1
5,010506010169,,E-MAIL_SERVICE_CAMP,astapovanata75@gmail.com,Saturday,17,1
6,010600038267,,ЦА-БКЛОЯЛЬНОСТЬ-1119,nellidobro@gmail.com,Thursday,17,2
7,010600571940,,ЦА-БИЗНДИР33-0320,aldarik@bk.ru,Wednesday,09,2
8,010601142207,,E-MAIL_SERVICE_CAMP,01ewgeni@mail.ru,Saturday,18,1
9,010603122213,,E-MAIL_SERVICE_CAMP,gs004@mail.ru,Saturday,20,1


In [50]:
best_time_sdf\
    .withColumn('email_cnt_dist', 
                    f.approx_count_distinct(f.col('email'))\
                    .over(Window.partitionBy('inn', 'kpp')).cast('integer'))\
    .filter("email_cnt_dist > 1").show()

,inn,kpp,campaign_nm,email,asdayname,ashour,day_open_cnt,email_cnt_dist
0,022000625129,,E-MAIL_SERVICE_CAMP,y-marsell@mail.ru,Saturday,15,1,2
1,022000625129,,E-MAIL_SERVICE_CAMP1,y-marsell@mail.ru,Wednesday,10,1,2
2,022000625129,,ЦА-БИЗНДИР33-0320,y_marsell@mail.ru,Wednesday,10,1,2
3,022000625129,,ЦА-БИЗНДИР34-0420,y-marsell@mail.ru,Monday,14,0,2
4,022501817488,,ЦА-БИЗНДИР34-0420,ilgizon_777@mail.ru,Tuesday,09,1,2
5,022501817488,,ЦА-ЦБКМАКИЕНКО-0420,i.muhametov@grandufa.ru,Saturday,22,1,2
6,023640641000,,ЦА-БИЗНДИР33-0320,tagevosyan74@list.ru,Monday,08,1,2
7,023640641000,,E-MAIL_SERVICE_CAMP1,tadevosyan74@list.ru,Tuesday,21,1,2
8,023640641000,,ЦА-КОРЭДО-0420,tadevosyan74@list.ru,Monday,10,1,2
9,023640641000,,ЦА-ПОБЕДА75БКМАЙЛ-0420,tadevosyan74@list.ru,Wednesday,13,1,2


## Select users with flag equals 1

In [51]:
strdate = datetime.strftime(datetime.now(), format='%Y.%d.%m %H:%M:%S')
currdate = datetime.strptime(strdate,'%Y.%d.%m %H:%M:%S')

In [52]:
# prev_load_dt_str

In [53]:
# daysback = 90

# mysql = \
# '''
# select 
# date_format(last_day(add_months(trunc(from_unixtime(unix_timestamp(min(last_open_time)) - {shiftd}*24*60*60),'MM'),-1)), 'yyyy-MM-dd') min_last_open_time_m90, 
# date_format(last_day(add_months(trunc(from_unixtime(unix_timestamp(min(last_click_time)) - {shiftd}*24*60*60), 'MM'),-1)), 'yyyy-MM-dd') min_last_click_time_m90
# from T_TEAM_SPEECH_ANALYTICS.RESP_INN_KPP_CAMP
# where load_dt > timestamp({dt})
# '''.format(shiftd=daysback, dt=prev_load_dt_str)

# min_clk_open = hive.sql(mysql).collect()

In [54]:
# min_last_open_time  = min_clk_open[0]['min_last_open_time_m90']
# min_last_open_time

In [55]:
sdf_fresh = sdf.filter((f.col('response_dttm') >= resp_dttm_min_str) & (f.col('fresh_user')==1)).select('inn', 
                                                                                                        'kpp', 
                                                                                                        'campaign_nm', 
                                                                                                        'email', 
                                                                                                        'fresh_user')
sdf_fresh = sdf_fresh.withColumn('email_proba',f.lit(0.0).cast(FloatType()))
sdf_fresh = sdf_fresh.withColumn('email_class',f.lit(-1).cast(IntegerType()))                                 
                                 
# sdf_fresh = hive.sql('''
# select
# distinct
#          inn
#         ,kpp
#         ,campaign_nm
#         ,email
#         ,cast(email_proba as float)
#         ,cast(email_class as int)
#         ,fresh_user
# from(
#     select 
#         inn 
#         ,kpp 
#         ,campaign_nm
#         ,email
#         ,0.0 email_proba
#         ,-1 email_class
#         ,fresh_user
#     from 
#     tmp_resp_inn_kpp_camp
#     where fresh_user = 1
#     and response_dttm >= timestamp('{dt}')
# )
# '''.format(dt=min_last_open_time))

In [56]:
sdf_fresh = sdf_fresh.withColumn('load_dt', f.lit(currdate).cast(TimestampType()))

In [57]:
sdf_fresh.cache()

DataFrame[inn: string, kpp: string, campaign_nm: string, email: string, fresh_user: int, email_proba: float, email_class: int, load_dt: timestamp]

In [58]:
sdf_fresh.count()

1391054

## Join with ML360

In [59]:
ml360 = 'dl_t_team_ml360'

In [60]:
sdf = hive.sql('''
with mon_products as
(
         select * from {tbl}.mon_products_20200414033929
         union select * from {tbl}.mon_products_20200629064208
         union select * from {tbl}.mon_products_20200703111831
         union select * from {tbl}.mon_products_20200707095006
)

select 
t.*,
concat_ws(' ',email_subject, message_title, 
            customer_text_03, customer_text_04, customer_text_05, 
            customer_text_06, customer_text_07, customer_text_08, customer_text_09, customer_text_10,
                              customer_text_12, customer_text_13, customer_text_14, customer_text_15, 
                              customer_text_17, customer_text_18, customer_text_19, customer_text_20) as text
from(
         select 
             rik.inn
            ,rik.kpp
            ,rik.campaign_nm
            ,rik.email
            --,rik.target            
            ,rik.organization_id
            ,rik.custid_value
            ,rik.response
            ,rik.response_dttm
            ,rik.asday
            ,rik.asdayname
            ,rik.ashour
            ,rik.astime           
            ,rik.email_subject
            ,rik.message_title
            ,rik.customer_text_01
            ,rik.customer_text_02
            ,rik.customer_text_03
            ,rik.customer_text_04
            ,rik.customer_text_05
            ,rik.customer_text_06
            ,rik.customer_text_07
            ,rik.customer_text_08
            ,rik.customer_text_09
            ,rik.customer_text_10
            ,rik.customer_text_11
            ,rik.customer_text_12
            ,rik.customer_text_13
            ,rik.customer_text_14
            ,rik.customer_text_15
            ,rik.customer_text_16
            ,rik.customer_text_17
            ,rik.customer_text_18
            ,rik.customer_text_19
            ,rik.customer_text_20
            ,rik.clk_on_dlv_all_rate_m11
            ,rik.clk_on_dlv_all_rate_m21
            ,rik.clk_on_dlv_all_rate_m31
            ,rik.clk_on_dlv_day_rate_m11
            ,rik.clk_on_dlv_day_rate_m21
            ,rik.clk_on_dlv_day_rate_m31
            ,rik.clk_on_open_all_rate_m11
            ,rik.clk_on_open_all_rate_m21
            ,rik.clk_on_open_all_rate_m31
            ,rik.clk_on_open_day_rate_m11
            ,rik.clk_on_open_day_rate_m21
            ,rik.clk_on_open_day_rate_m31
            ,rik.delivery_all_cnt_m11
            ,rik.delivery_all_cnt_m21
            ,rik.delivery_all_cnt_m31
            ,rik.diff_fcl_fo_m11
            ,rik.diff_fcl_fo_m21
            ,rik.diff_fcl_fo_m31
            ,rik.diff_fcl_fs_m11
            ,rik.diff_fcl_fs_m21
            ,rik.diff_fcl_fs_m31
            ,rik.diff_fcl_lo_m11
            ,rik.diff_fcl_lo_m21
            ,rik.diff_fcl_lo_m31
            ,rik.diff_fcl_ls_m11
            ,rik.diff_fcl_ls_m21
            ,rik.diff_fcl_ls_m31
            ,rik.diff_fo_fs_m11
            ,rik.diff_fo_fs_m21
            ,rik.diff_fo_fs_m31
            ,rik.diff_fo_ls_m11
            ,rik.diff_fo_ls_m21
            ,rik.diff_fo_ls_m31
            ,rik.diff_lcl_fcl_m11
            ,rik.diff_lcl_fcl_m21
            ,rik.diff_lcl_fcl_m31
            ,rik.diff_lcl_fo_m11
            ,rik.diff_lcl_fo_m21
            ,rik.diff_lcl_fo_m31
            ,rik.diff_lcl_fs_m11
            ,rik.diff_lcl_fs_m21
            ,rik.diff_lcl_fs_m31
            ,rik.diff_lcl_lo_m11
            ,rik.diff_lcl_lo_m21
            ,rik.diff_lcl_lo_m31
            ,rik.diff_lcl_ls_m11
            ,rik.diff_lcl_ls_m21
            ,rik.diff_lcl_ls_m31
            ,rik.diff_lo_fo_m11
            ,rik.diff_lo_fo_m21
            ,rik.diff_lo_fo_m31
            ,rik.diff_lo_fs_m11
            ,rik.diff_lo_fs_m21
            ,rik.diff_lo_fs_m31
            ,rik.diff_lo_ls_m11
            ,rik.diff_lo_ls_m21
            ,rik.diff_lo_ls_m31
            ,rik.first_click_time
            ,rik.first_click_time_m11
            ,rik.first_click_time_m21
            ,rik.first_click_time_m31
            ,rik.first_open_time
            ,rik.first_open_time_m11
            ,rik.first_open_time_m21
            ,rik.first_open_time_m31
            ,rik.first_send_time
            ,rik.first_send_time_m11
            ,rik.first_send_time_m21
            ,rik.first_send_time_m31
            ,rik.fresh_user
            ,rik.last_click_all_cnt_m11
            ,rik.last_click_all_cnt_m21
            ,rik.last_click_all_cnt_m31
            ,rik.last_click_day_cnt_m11
            ,rik.last_click_day_cnt_m21
            ,rik.last_click_day_cnt_m31
            ,rik.last_click_time
            ,rik.last_click_time_m11
            ,rik.last_click_time_m21
            ,rik.last_click_time_m31
            ,rik.last_open_all_cnt_m11
            ,rik.last_open_all_cnt_m21
            ,rik.last_open_all_cnt_m31
            ,rik.last_open_day_cnt_m11
            ,rik.last_open_day_cnt_m21
            ,rik.last_open_day_cnt_m31
            ,rik.last_open_time
            ,rik.last_open_time_m11
            ,rik.last_open_time_m21
            ,rik.last_open_time_m31
            ,rik.last_send_time
            ,rik.last_send_time_m11
            ,rik.last_send_time_m21
            ,rik.last_send_time_m31
            ,rik.open_on_dlv_all_rate_m11
            ,rik.open_on_dlv_all_rate_m21
            ,rik.open_on_dlv_all_rate_m31
            ,rik.open_on_dlv_day_rate_m11
            ,rik.open_on_dlv_day_rate_m21
            ,rik.open_on_dlv_day_rate_m31
            ,rik.sum_open_click_camp
            ,rik.load_dt
            ,rik.last_day

            ,mcs.cred_sdo
            ,mcs.weight_prc
            ,mcs.rate_diff

            ,mdh.all_prod_deals_created
            ,mdh.all_prod_deals_completed
            ,mdh.credit_prod_deals_created
            ,mdh.merch_prod_deals_created
            ,mdh.corp_cards_prod_deals_created
            ,mdh.salary_prod_deals_created
            ,mdh.depozit_prod_deals_created
            ,mdh.insure_prod_deals_created
            ,mdh.credit_prod_deals_completed
            ,mdh.merch_prod_deals_completed
            ,mdh.corp_cards_prod_deals_completed
            ,mdh.salary_prod_deals_completed
            ,mdh.depozit_prod_deals_completed
            ,mdh.insure_prod_deals_completed
            --,mdh.segment

            ,mibs.complicity_type
            ,mibs.otrasl
            ,cast(mibs.ul_kopf_cd as int) ul_kopf_cd
            ,mibs.big_segment

            ,mip.oborot_kredit
            ,mip.invest_kredit
            ,mip.overdraft_kredit
            ,mip.proekt_finans
            ,mip.products_lizing
            ,mip.obsluzh_rts_rur
            ,mip.obsluzh_rts_val
            ,mip.spec_acc
            ,mip.deposit
            ,mip.veksel
            ,mip.deposit_sertificat
            ,mip.inkass
            ,mip.samoinkass
            ,mip.zarplat_projects
            ,mip.acquiring_torg
            ,mip.acquiring_mobile
            ,mip.acreditiv
            ,mip.valuta_control
            ,mip.garantee_kontract
            ,mip.garantee_gos
            ,mip.garantee_other
            ,mip.arenda_seyf
            ,mip.dbo
            ,mip.corporate_cards
            ,mip.cash_management
            ,mip.bank_straxovanie_imush
            ,mip.bank_straxovanie_lich
            ,mip.bank_straxovanie_otvet
            ,mip.einvoicing
            ,mip.factoring
            ,mip.konversion
            ,mip.ns_servis
            ,mip.ns_terminal

            ,mnbc.segment crm_segment
            ,mnbc.prior_true
            ,mnbc.OPF_IP
            ,mnbc.OPF_OOO
            ,mnbc.OPF_Other
            ,mnbc.tb_Centralno_Chernozemnyj
            ,mnbc.tb_Moskovskij
            ,mnbc.tb_Uralskij
            ,mnbc.tb_Povolzhskij
            ,mnbc.tb_Severo_Zapadnyj
            ,mnbc.tb_Dalnevostochnyj
            ,mnbc.tb_Volgo_Vyatskij
            
            ,mp.corp_cards
            ,mp.rko
            ,mp.salary
            ,mp.merch
            ,mp.credit
            ,mp.deposits            

            ,yfp.egr_org_id
            ,yfp.ip_flg
            ,yfp.ul_org_id
            ,yfp.egrul_org_id
            ,yfp.ogrn
            ,yfp.short_nm
            ,yfp.okato_cd
            ,yfp.kopf_cd
            ,yfp.kopf_nm
            ,cast(yfp.active_flg as int) active_flg
            ,yfp.founders_all_cnt
            ,yfp.founders_ul_cnt
            ,yfp.head_inn
            ,yfp.founders_fl_cnt
            ,yfp.founders_foreign_ul_cnt
            ,yfp.branch_cnt
            ,yfp.adrs_index
            ,yfp.adrs_rf_subject
            ,yfp.egrip_org_id
            ,yfp.ip_org_id
            ,yfp.ar_revenue
            ,yfp.ar_total_expenses
            ,yfp.ar_profit_before_tax
            ,yfp.ar_taxes
            ,yfp.ar_net_profit
            ,yfp.fot_balance
            ,yfp.license_cnt
            ,yfp.okfs_cd
            ,yfp.integrum_staff_range
            ,yfp.ab_immobilized_assets
            ,yfp.ab_losses
            ,yfp.ab_own_capital
            ,yfp.ab_borrowed_capital
            ,yfp.okved_cnt
            ,yfp.industry
            ,yfp.total_sum_kt
            ,yfp.total_count_kt
            ,yfp.total_sum_dt
            ,yfp.total_count_dt
            ,yfp.total_sum_kt_3m
            ,yfp.total_count_kt_3m
            ,yfp.total_sum_dt_3m
            ,yfp.total_count_dt_3m
            ,yfp.total_sum_kt_year18
            ,yfp.total_count_kt_year18
            ,yfp.total_sum_dt_year18
            ,yfp.total_count_dt_year18
            ,yfp.tax_income
            ,yfp.penalty_income
            ,yfp.tax_ip
            ,yfp.penalty_ip
            ,yfp.tax_foreign
            ,yfp.penalty_foreign
            ,yfp.tax_income_3m
            ,yfp.penalty_income_3m
            ,yfp.tax_ip_3m
            ,yfp.penalty_ip_3m
            ,yfp.tax_foreign_3m
            ,yfp.penalty_foreign_3m
            ,yfp.tax_income_year18
            ,yfp.penalty_income_year18
            ,yfp.tax_ip_year18
            ,yfp.penalty_ip_year18
            ,yfp.tax_foreign_year18
            ,yfp.penalty_foreign_year18
            ,yfp.kpp_regions_cnt
            ,yfp.gis_building_area
            ,yfp.gis_merchant_cnt
            ,yfp.gis_staff_cnt
            ,yfp.gis_building_firms
            ,yfp.gis_network_flg
            ,yfp.gis_rosn_merchant_flg
            ,yfp.gis_opt_merchant_flg
            ,yfp.gis_production_merchant_flg
            ,yfp.gis_internet_merchant_flg
            ,yfp.gis_bank_payment_flg
            ,yfp.gis_cash_payment_flg
            ,yfp.gis_card_payment_flg
            ,yfp.gis_internet_payment_flg
            ,yfp.cred_application_cnt
            ,yfp.segment fot_segment
            ,yfp.okato_reg
            ,yfp.integrum_lower_bound
            ,yfp.wlt_prediction
            ,yfp.fns_staff_cnt
            ,yfp.mean_zp
            ,yfp.zp_amount
            ,yfp.mean_zp_empl_cnt
            ,yfp.max_zp_empl_cnt
            ,yfp.min_zp_empl_cnt
            ,yfp.stoplist

         from tmp_resp_inn_kpp_camp    rik
         left join {tbl}.mon_cred_sdo                mcs  on rik.inn = mcs.inn  and rik.last_day = mcs.mon
         left join {tbl}.mon_deals_history           mdh  on rik.inn = mdh.inn  and rik.last_day = mdh.mon
         left join {tbl}.mon_iskra_big_segment       mibs on rik.inn = mibs.inn and rik.last_day = mibs.mon
         left join {tbl}.mon_iskra_prod              mip  on rik.inn = mip.inn  and rik.last_day = mip.mon
         left join {tbl}.mon_nba_base_crm            mnbc on rik.inn = mnbc.inn and rik.last_day = mnbc.mon
         left join mon_products                      mp   on rik.inn = mp.inn   and rik.last_day = mp.mon  
         left join {tbl}.yr_fot_profile              yfp  on rik.inn = yfp.inn  and rik.last_day = yfp.yr  
                                                                                and rik.kpp = yfp.kpp      
         where fresh_user in (0,2)
         --and load_dt <= timestamp('%s')
) t
--where target in (0,1)
'''.format(tbl=ml360)) #%(prev_load_dt_str))

In [61]:
# best_day_sdf = sdf.distinct()

### Join with CALL_TASK_TO_SENDSAY

In [62]:
call_sdf = hive.sql("select cast( cast(inn as long) as string) _inn, CREATE_DATE from sbx_team_digitcamp.CALL_TASK_TO_SENDSAY")

In [63]:
call_sdf.dtypes

[('_inn', 'string'), ('CREATE_DATE', 'timestamp')]

In [64]:
sdf = sdf.join(call_sdf, on=( (sdf.inn==call_sdf._inn) & 
                              (call_sdf.CREATE_DATE >= f.date_sub(sdf.first_send_time, 40))))

In [65]:
sdf = sdf.withColumn('call_first_check', f.when((f.col('CREATE_DATE') < f.col('last_send_time')) & \
                                                (f.col('CREATE_DATE') >= f.date_sub(f.col('last_send_time'),30)), 1).otherwise(0))

In [ ]:
sdf.cache()

In [67]:
sdf.count()

34287783

### Preproc

In [70]:
sdf = sdf.withColumn('idx', f.monotonically_increasing_id())

In [71]:
import locale
import re
from functools import cmp_to_key

# indx_cols = list(zip(range(len(final_cols)),final_cols))

customer_attr = ['customer_text_0{}'.format(str(i)) if i<10 
                 else 'customer_text_{}'.format(str(i)) for i in range(1,21)]

first_order_cols = \
[
    'inn',
    'kpp', 
    'organization_id',
    'campaign_nm',
    'email',
    #'target'
]

rem_cols = \
[
#'sum_open_click_camp', 
'custid_value', 
'CREATE_DATE',
'_inn',
'response', 'asday','astime', 'ashour', 'asdayname', 'yr',
'email_subject', 'message_title',
'wlt_prediction',
'adrs_index', 'adrs_rf_subject', 'head_inn', 'kopf_nm', 'short_nm',
'last_day',
'mon', 'okato_cd', 'kopf_cd','okfs_cd','okved_cnt','egrip_org_id','ip_org_id','integrum_staff_range',
'gis_building_area','gis_staff_cnt','gis_building_firms','okato_reg','fns_staff_cnt',
'load_dt', 'obsluzh_rts_rur','salary','corp_cards', 'ogrn'
]


reordered_cols = first_order_cols+ \
sorted(set(sdf.columns) - set(customer_attr)
                        - set(first_order_cols)
                        - set(rem_cols), key=cmp_to_key(locale.strcoll))

pattern = re.compile(r'.*_click_time.*|.*_open_time.*|.*_send_time.*|delivery_all_cnt.*')
# 'delivery_all_cnt_m11','delivery_all_cnt_m21'

final_cols = [col for col in reordered_cols if len(pattern.findall(col)) == 0]

sdf = sdf.select(*final_cols)

### Fill NAN

In [72]:
replna = {}
for col in sdf.dtypes:
    if col[1] == 'string':
        replna.update({col[0]:''})
    elif col[1] == 'boolean':
        replna.update({col[0]:False})
    elif col[1] != 'timestamp':
        replna.update({col[0]:0})

sdf = sdf.fillna(replna)

In [73]:
# sdf = sdf.withColumn('active_flg', sdf['active_flg'].cast(IntegerType()))
# sdf = sdf.withColumn('ul_kopf_cd', f.col('ul_kopf_cd').cast(IntegerType()))

In [74]:
for col in sdf.dtypes:
    if col[1] == 'string':
        print(col)

('inn', 'string')
('_inn', 'string')
('kpp', 'string')
('organization_id', 'string')
('campaign_nm', 'string')
('email', 'string')
('big_segment', 'string')
('complicity_type', 'string')
('crm_segment', 'string')
('fot_segment', 'string')
('otrasl', 'string')
('prior_true', 'string')
('text', 'string')


### Fit spark dataframe: index categorical columns

In [75]:
#catcol = ['big_segment', 'complicity_type', 'otrasl', 'prior_true', 'segment_main'] 

# indexers = [StringIndexer(inputCol=column, outputCol=column+'_idx').setHandleInvalid('keep').fit(sdf) for column in catcol]
# pipelineModel = Pipeline(stages=indexers)
# pipelineModelFit = pipelineModel.fit(sdf)
# sdf = pipelineModelFit.transform(sdf)

## pipeline save StringIndexer
# pipelineModelFit.write().overwrite().save("/user/ektov/sandsay_pipeline_StringIndexer")

### Load Prefitted PipelineModel

In [76]:
# ! hdfs dfs -cp -f hdfs://nameservice1/user/ektov/sendsay_pipeline_StringIndexer_upd hdfs:///user/ektov/

In [77]:
pipelineModelPreFit = PipelineModel.load("hdfs:///user/{}/sendsay_pipeline_StringIndexer_upd".format(curruser))
sdf = pipelineModelPreFit.transform(sdf)

# drop string columns
catcol = ['big_segment', 'crm_segment', 'fot_segment', 'complicity_type', 'otrasl', 'prior_true'] 
catcol.extend(['response_dttm'])
sdf = sdf.drop(*catcol)

### Remove constant columns

In [78]:
# const_cols = sdf.agg(*[f.countDistinct(f.col(col)).alias(col) for col in sdf.columns]).collect()[0].asDict()
# cols_to_rem = sorted([k for k,v in const_cols.items() if v == 0])

In [79]:
with open('./csv/zero_std_cols', 'r') as fin:
    res = fin.readlines()
zero_std_cols = [re.sub('\\n','',line) for line in res]

In [80]:
sdf = sdf.drop(*zero_std_cols)

### Save

In [81]:
# sp.db = 't_team_speech_analytics'
# sp.output = 'data_temp_model_sendsay'
# sp.save(sdf_exp)

In [82]:
# sdf = hive.sql(''' 
# select
# *
#  from T_TEAM_SPEECH_ANALYTICS.data_temp_model_sendsay
# ''')

## Preprocessing with FastText

In [83]:
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize

import pymorphy2
# morph = pymorphy2.MorphAnalyzer()
# broadcast 
MORPH = pymorphy2.MorphAnalyzer()
morph_broadcast = sp.sc.broadcast(MORPH)

stop_words = set(stopwords.words('russian'))
exclude = set(string.punctuation) 

def text_prepare(text, lemmatize = False): 
    morph = morph_broadcast.value
    def drop_nps(tokenslst, lemmatize=lemmatize):
        lres=[]
        for token in tokenslst:
            parse_res=morph.parse(token)[0]
            if not 'Name' in parse_res.tag and \
               not 'Patr' in parse_res.tag and \
               not 'Surn' in parse_res.tag:
                        if not lemmatize:
                            lres.append(parse_res.word)
                        else:
                            lres.append(parse_res.normal_form)      
        return ' '.join(lres)

    text = ''.join(x if x not in exclude else ' ' for x in text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    text = re.sub(r'[a-zA-Z\d{1,8}]+','',text)
    text = ' '.join(word for word in text.split() if len(word)>2) 
    tokenslst = list(map(lambda x: x.replace('"', '').lower(), RegexpTokenizer('([а-яА-Яa-zA-Z\d{9,12}]+|\"[\w\s]+\")').tokenize(text)))

    if not lemmatize:
        res = drop_nps(tokenslst)
    else:    
        res = drop_nps(tokenslst, True)    
    return res

In [84]:
text_prepare_udf = f.udf(text_prepare, StringType())

In [85]:
sdf = sdf.withColumn('text_norm', text_prepare_udf('text'))  

### TFIDF+FastText: Run broadcast 

In [ ]:
from pyspark import SparkFiles

In [87]:
# !hdfs dfs -put /home/ektov/notebooks/EmailSMSResponses/RESP_MODELS/model/email_sendsay_TfIdf_vect.pkl hdfs:///user/ektov/

In [88]:
# filemodel = 'model/'
# model_ft =  joblib.load(os.path.join(filemodel,'email_sendsay_FT_gensi_vocabulary.pkl'))
# embed = joblib.load(os.path.join(filemodel,'email_sendsay_TfIdf_vect.pkl'))

hdfsfile='hdfs:///user/{}/email_sendsay_TfIdf_vect.pkl'.format(curruser)
sp.sc.addFile(hdfsfile)
embed = joblib.load(SparkFiles.get('email_sendsay_TfIdf_vect.pkl'))

In [89]:
%%time
# bstBroadcast = sp.sc.broadcast(model_ft)
# vectBroadcast = sp.sc.broadcast(model_idf)
rev_dct = sp.sc.broadcast(embed.gensim_rev_dct)
tfIdfModel = sp.sc.broadcast(embed.gensim_tfIdfModel)
bow = sp.sc.broadcast(embed.bow)
model_word_tfidf = sp.sc.broadcast(embed.model_word_tfidf)
model_char_tfidf = sp.sc.broadcast(embed.model_char_tfidf)

col_bc = sp.sc.broadcast(sdf.columns)

CPU times: user 9.11 ms, sys: 3.37 ms, total: 12.5 ms
Wall time: 78 ms


In [90]:
colexp = col_bc.value
colexp.remove('text')
colexp.remove('text_norm')

In [91]:
sp.sc.addFile('./../src/corpora_transform_main.py')


#!hdfs dfs -put model/ft_vector /user/ektov/
hdfsfile = 'hdfs:///user/{}/FT/ft_vector'.format(curruser)
sp.sc.addFile(hdfsfile)

# !hdfs dfs -put model/ft_vector.vectors_ngrams.npy /user/ektov/
# hdfsfile = 'hdfs:///user/ektov/ft_vector.vectors_ngrams.npy'
# sp.sc.addFile(hdfsfile)

#!hdfs dfs -put model/ft.word2vec /user/ektov/
hdfsfile = 'hdfs:///user/{}/FT/ft.word2vec'.format(curruser)
sp.sc.addFile(hdfsfile)

___

In [92]:
def block_iterator(iterator,size):
    """
    dest:
        сервисная функция для итерации по блокам данных внутри RDD. 
        Чем больше size при увеличении количества потоков, тем быстрее обработка
    args:
        iterator-объект 
        size - размер элементов для единичной итерации
    return:
        вычисляемый объект bucket
 
    """
    bucket = list()
    for e in iterator:
        bucket.append(e)
        if len(bucket) >= size:
            yield bucket
            bucket = list()
    if bucket:
        yield bucket

### TfIdfFastTextEmbedVectorizer: Generate Features Based on FastText Vector Only

In [93]:
%%time
def block_classify(iterator):
    curruser = os.environ.get('USER')
    if isUseOptWorkspace:
#         sys.path.insert(0,'/home/{}/.local/lib/python3.5/site-packages/'.format(curruser))
        sys.path.insert(0,'/opt/workspace/{}/libs/python3.5/site-packages'.format(curruser))
    else:
        sys.path.insert(0,'/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    from corpora_transform_main import TfIdfFastTextEmbedVectorizer
    from gensim.models.fasttext import FastText as FT_gensim
    from gensim.models import KeyedVectors
    import pandas as pd
    import json

    for features in block_iterator(iterator, 1000000):
        
        features_df = pd.DataFrame(list(features), columns = col_bc.value) # получаем исходные данные в виде df        
#         X_predict = features_df[[col for col in features_df.columns if col not in ['inn', 'kpp', 'campaign_nm', 'text']]]    
        X_predict = features_df['text_norm']
    
#         model_ft = FT_gensim.load(SparkFiles.get('ft_vector'))
        model_ft = KeyedVectors.load_word2vec_format(SparkFiles.get('ft.word2vec'))
        _useTfIdf2Pred = False
        embed = TfIdfFastTextEmbedVectorizer(model = model_ft, 
                                             useTfIdf2Pred=_useTfIdf2Pred,
                                             word_ngram_range=(1,2), 
                                             word_max_vec_features=300)
        
        embed.gensim_rev_dct = rev_dct.value 
        embed.gensim_tfIdfModel = tfIdfModel.value
        embed.bow = bow.value
        
        if _useTfIdf2Pred:
            embed.model_word_tfidf = model_word_tfidf.value
            embed.model_char_tfidf = model_char_tfidf.value

        res = embed.transform(X_predict)
        ft_df = pd.DataFrame(res, columns=embed.feature_names)
        
        res_df = features_df[colexp]
        res_df = pd.concat([res_df,ft_df], axis=1)
 
        for e in json.loads(res_df.to_json(orient='records')): # генерируем выходной блок
            yield e
        
            
sdf_proc = sdf.rdd.mapPartitions(block_classify)

CPU times: user 556 µs, sys: 373 µs, total: 929 µs
Wall time: 271 ms


In [94]:
rdd  = sdf.limit(1).rdd.mapPartitions(block_classify)
flat = rdd.flatMap(lambda x: [key for key in x.keys()]).collect()

In [95]:
ft_cols = [key for key in flat if '_ft_' in key.lower()]

typesmap_rdd={}

for column_name, column in sdf.dtypes:
    if column == 'string':
        typesmap_rdd[column_name] = StringType()
    elif 'decimal' in column:
#         digits = int('{}'.format(column.split('(')[1].split(',')[0]))
#         prec   = int('{}'.format(column.split('(')[1].split(',')[1][:-1]))
        typesmap_rdd[column_name] = FloatType() #DecimalType(digits,prec)
    elif column == 'double':
        typesmap_rdd[column_name] = DoubleType()        
    elif column == 'float':
        typesmap_rdd[column_name] = FloatType()
    elif column == 'int':
        typesmap_rdd[column_name] = IntegerType()       
    elif column == 'bigint':
        typesmap_rdd[column_name] = LongType()   

for column_name in ft_cols:        
       typesmap_rdd[column_name] = FloatType() 

In [96]:
cols = colexp+ft_cols

currschema = StructType([StructField(col, typesmap_rdd[col]) for col in cols])

sdf_proc = sp.sql.createDataFrame(sdf_proc, schema=currschema)

In [97]:
# sdf_proc.cache()

In [98]:
# sdf_proc.count()

## Load PreTrained Xgboost model

In [ ]:
from getpass import getpass
from sshloader import Ssh

In [100]:
# USERNAME = "ektov"
# PASSWORD = getpass()

In [101]:
# _ssh = Ssh(username=USERNAME, 
#            password=PASSWORD, 
#            host='februs11.ca.sbrf.ru',
#            verbosity=1)
# _ssh.setup()  

In [102]:
# rpath = '/home/{user}/notebook/sendsay_model/model/'.format(user=USERNAME)
# lpath = './model/'
# file = 'xgboost_only_ft_200207.pkl'
# _ssh.getFiles(rpath,lpath,file)

In [100]:
import xgboost
from xgboost import XGBClassifier

file = 'xgb_best_param_wFT_10M_200613.pkl'
model = joblib.load('./pkl/{}'.format(file))

### Make a Prediction: Run broadcast

In [110]:
%%time
bstBroadcast = sp.sc.broadcast(model)
col_bc = sp.sc.broadcast(sdf_proc.columns)

def block_classify(iterator):
    import os
    curruser = os.environ.get('USER')
    if isUseOptWorkspace:
#         sys.path.insert(0,'/home/{}/.local/lib/python3.5/site-packages/'.format(curruser))
        sys.path.insert(0,'/opt/workspace/{}/libs/python3.5/site-packages'.format(curruser))
    else:
        sys.path.insert(0,'/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    import xgboost
    import pandas as pd
    import json
    import re
    import ast

    for features in block_iterator(iterator, 500000):
        
        features_df = pd.DataFrame(list(features), columns = col_bc.value) # получаем исходные данные в виде df 
        
#         pattern = re.compile(r'diff_fo_fs.*|diff_lo_fs.*|diff_fcl_fo.*|diff_fcl_fs.*|diff_lcl_fo.*|diff_lcl_fs.*|diff_fcl_lo.*|.*_day_.*|corp_cards_prod_deals.*')

        delcols = \
         ['inn', 
          'kpp', 
          'campaign_nm',
          'email',
          'organization_id', 
          'idx', 
          'fresh_user',  
          'response_dttm'
          'target',
          'sum_open_click_camp', 
          ]         
        
#         filter_cols = [col for col in features_df.columns if (len(pattern.findall(col)) == 0) and (col not in delcols)]
        filter_cols = [col for col in features_df.columns if (col not in delcols)]
#         sortcols = sorted(filter_cols)
#         X_predict = features_df[sortcols]
        X_predict = features_df[filter_cols]

        try:
            predicted_proba = bstBroadcast.value.predict_proba(X_predict.astype(float))[:,1]
            predicted       = bstBroadcast.value.predict(X_predict.astype(float))
        except ValueError as err:
            colsstr = str(err).split('feature_names mismatch: ')[1].split(' [')[0]
            getcolslist = list(ast.literal_eval(colsstr[1:-1]))
            X_predict = features_df[getcolslist]       
            predicted_proba = bstBroadcast.value.predict_proba(X_predict.astype(float))[:,1]
            predicted       = bstBroadcast.value.predict(X_predict.astype(float))

        res_df                = features_df[['campaign_nm']]
        res_df["email"]       = features_df[['email']]
        res_df["inn"]         = features_df[['inn']]
        res_df["kpp"]         = features_df[['kpp']]
        res_df["email_proba"] = predicted_proba
        res_df["email_class"] = predicted
        res_df["fresh_user"]  = features_df[['fresh_user']]
        res_df = res_df[['inn','kpp', 'email', 'campaign_nm','email_proba','email_class', 'fresh_user']]

        for e in json.loads(res_df.to_json(orient='records')): # генерируем выходной блок
            yield e
            
scores = sdf_proc.rdd.mapPartitions(block_classify).toDF()

CPU times: user 76.7 ms, sys: 62.3 ms, total: 139 ms
Wall time: 1min 46s


In [114]:
scores.count()

24678186

### Add some specific columns

In [115]:
scores = scores.select([f.col('inn'),
                        f.col('kpp'),
                        f.col('campaign_nm'),
                        f.col('email'),
                        f.col('email_proba').cast(FloatType()),
                        f.col('email_class').cast(IntegerType()),
                        f.col('fresh_user')])
scores = scores.withColumn('load_dt', f.lit(currdate).cast(TimestampType()))

## Join scored users with inactive subset

In [116]:
sdf_fresh = sdf_fresh[scores.columns]

In [117]:
fin_sdf = scores.distinct().union(sdf_fresh)

## Agregating by `INN` `KPP` `CAMP` `EMAIL` and keep only best probabilities

In [ ]:
# fin_sdf = \
# fin_sdf.withColumn('max_email_proba', 
#                     f.max('email_proba')\
#                     .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')))\
#     .withColumn('pos', f.when(f.col('email_proba') - f.col('max_email_proba') == 0.0, 1).otherwise(0))\

In [ ]:
# fin_sdf = fin_sdf.filter('pos == 1').select('inn', 
#                                             'kpp', 
#                                             'campaign_nm', 
#                                             'email', 
#                                             'email_proba', 
#                                             'email_class', 
#                                             'fresh_user', 
#                                             'load_dt')

In [118]:
fin_sdf = fin_sdf\
         .withColumn('top_proba', f.dense_rank().over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')\
                                                .orderBy(f.col('email_proba').desc())))\
                                                .filter('''top_proba = 1''') 

In [119]:
# fin_sdf = fin_sdf.rdd.sortBy(lambda x: (x[0], x[1], x[2], x[3], x[4]), ascending=False).toDF()
fin_sdf = fin_sdf.groupBy(['inn', 'kpp', 'campaign_nm', 'email']).agg(f.first(f.col('email_proba')).alias('email_proba'), 
                                                                      f.first(f.col('email_class')).alias('email_class'),
                                                                      f.first(f.col('fresh_user')).alias('fresh_user'),
                                                                      f.first(f.col('load_dt')).alias('load_dt'))

In [120]:
fin_sdf = fin_sdf.select('inn', 
                         'kpp', 
                         'campaign_nm', 
                         'email', 
                         'email_proba', 
                         'email_class', 
                         'fresh_user', 
                         'load_dt')

In [121]:
fin_sdf.filter('email_class = 1').show(10)

,inn,kpp,campaign_nm,email,email_proba,email_class,fresh_user,load_dt
0,1001208660,100101001,ЦА-БИЗНДИР33-0320,ovchinnikov71082@gmail.com,0.511,1,2,2020-07-27 09:28:29
1,1021503920,102101001,ЦА-БИЗНДИР33-0320,polinastarostina98@gmail.com,0.642,1,2,2020-07-27 09:28:29
2,110116098040,,E-MAIL_SERVICE_CAMP1,dkbushuev@gmail.com,0.607,1,0,2020-07-27 09:28:29
3,1103044393,110301001,ЦА-БИЗНДИР33-0320,kaskadik2009@mail.ru,0.512,1,2,2020-07-27 09:28:29
4,121803774200,,E-MAIL_SERVICE_CAMP,rumas121297@gmail.com,0.582,1,0,2020-07-27 09:28:29
5,1402022278,140201001,ЦА-БИЗНДИР33-0320,djudi83@inbox.ru,0.540,1,2,2020-07-27 09:28:29
6,143321159540,,E-MAIL_SERVICE_CAMP,shkazbek@gmail.com,0.585,1,0,2020-07-27 09:28:29
7,1615014673,161501001,ЦА-БИЗНДИР33-0320,artyom.kozhemiakin@tocteam.com,0.560,1,2,2020-07-27 09:28:29
8,1645028426,164501001,ЦА-БИЗНДИР33-0320,opt3@s-tris.ru,0.806,1,2,2020-07-27 09:28:29
9,1648005931,164801001,E-MAIL_SERVICE_CAMP,anisimow-14@mail.ru,0.575,1,0,2020-07-27 09:28:29


In [123]:
fin_sdf.select('inn','kpp').distinct().count()

1431936

In [124]:
fin_sdf = fin_sdf.select(*[f.col(col).alias(col+'_') for col in fin_sdf.columns])

## Join Scores with Best Day/Hour table

In [126]:
best_time_sdf.show()

,inn,kpp,campaign_nm,email,asdayname,ashour,day_open_cnt
0,010407628324,,ЦА-БИЗНДИР34ПОС-0420,tehnomir01@mail.ru,Tuesday,15,1
1,0105001202,010501001,E-MAIL_SERVICE_CAMP,9284666801@mail.ru,Monday,08,1
2,0105010398,010501001,E-MAIL_SERVICE_CAMP,arina0605@gmail.com,Friday,12,3
3,0105012701,010501001,E-MAIL_SERVICE_CAMP,tdmnormal@mail.ru,Saturday,01,0
4,0105021640,010501001,E-MAIL_SERVICE_CAMP,miriadakkm@mail.ru,Sunday,18,1
5,010506010169,,E-MAIL_SERVICE_CAMP,astapovanata75@gmail.com,Saturday,17,1
6,010600038267,,ЦА-БКЛОЯЛЬНОСТЬ-1119,nellidobro@gmail.com,Thursday,17,2
7,010600571940,,ЦА-БИЗНДИР33-0320,aldarik@bk.ru,Wednesday,09,2
8,010601142207,,E-MAIL_SERVICE_CAMP,01ewgeni@mail.ru,Saturday,18,1
9,010603122213,,E-MAIL_SERVICE_CAMP,gs004@mail.ru,Saturday,20,1


In [147]:
conditions= ((fin_sdf.inn_==best_time_sdf.inn)&
             (fin_sdf.kpp_==best_time_sdf.kpp)&
             (fin_sdf.campaign_nm_==best_time_sdf.campaign_nm)&
             (fin_sdf.email_==best_time_sdf.email))

res = fin_sdf.join(best_time_sdf, on=conditions, how='left_outer')

In [148]:
res = res.drop(*['inn', 'kpp', 'campaign_nm', 'email'])

In [149]:
res = res.drop_duplicates()

### !!!CREATE: DO THIS FOR THE FIRST TIME: Create main table 

In [153]:
res = res.select([f.col(col).alias(re.sub(r'\_$','',col)) for col in res.columns])

In [155]:
cols=\
['inn',
 'kpp',
 'campaign_nm',
 'email',
 'email_proba',
 'email_class',
 'asdayname',
 'ashour',
 'day_open_cnt',
 'fresh_user',
 'load_dt'
]
res = res.select(*cols)

In [156]:
table_name = 'RESP_INN_KPP_CAMP_EMAIL_PROBA'
tmp_tbl = 'tmp_{}'.format(table_name)

res.registerTempTable(tmp_tbl)
hive.cacheTable(tmp_tbl)

hive.sql('''drop table if exists sbx_team_digitcamp.{}__'''.format(table_name))
hive.sql('''create table sbx_team_digitcamp.{}__ select * from {}'''.format(table_name,tmp_tbl))
hive.sql('''drop table if exists sbx_team_digitcamp.{}'''.format(table_name))
subprocess.call(['hdfs', 'dfs', '-rm', '-R', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format(table_name.lower())])
hive.sql('''alter table sbx_team_digitcamp.{}__ rename to sbx_team_digitcamp.{}'''.format(table_name, table_name))
hive.sql("drop table if exists {}".format(tmp_tbl))

DataFrame[]

### !!!UPDATE: FOR EACH FOLLOWING CASES: Perform Full Oter Join with `RESP_INN_KPP_CAMP_EMAIL_PROBA`

In [ ]:
res = res.select([f.col(col).alias(re.sub(r'\_$','',col)) for col in res.columns])

In [107]:
res.registerTempTable('tmp_scores')
hive.cacheTable('tmp_scores')

In [113]:
import subprocess

In [157]:
sdf_resp = hive.sql("select * from sbx_team_digitcamp.RESP_INN_KPP_CAMP_EMAIL_PROBA")
sdf_resp.show()

,inn,kpp,campaign_nm,email,email_proba,email_class,asdayname,ashour,day_open_cnt,fresh_user,load_dt
0,0101009190,010101001,ЦА-БИЗНДИР35-0520,medfarma50@mail.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
1,010400038421,,ЦА-БИЗНДИР34-0420,29191@mail.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
2,0105012701,010501001,E-MAIL_SERVICE_CAMP,tdmnormal@mail.ru,0.000,-1,Saturday,01,0.000,1,2020-07-27 09:28:29
3,010502173469,,ЦА-БИЗНДИР35-0520,01natali@inbox.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
4,0105030411,010501001,ЦА-БИЗНДИР35-0520,ukk-maikopskiy2014@yandex.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
5,0105050827,010501001,ЦА-БИЗНДИР34-0420,transservis06@mail.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
6,0105051059,010501001,ЦА-БИЗНДИР35-0520,stulskij@mail.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
7,0106008761,010701001,ЦА-БИЗНДИР35-0520,info@ekspress-kuban.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
8,010706416021,,ЦА-БИЗНДИР35ПОС-0520,katrich2494@mail.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29
9,010801545411,,ЦА-БИЗНДИР35ПОС-0520,iren_pani@mail.ru,0.000,-1,None,None,nan,1,2020-07-27 09:28:29


### Check if we have more than 1 email per `campaign_nm` with different predicted scores

In [158]:
sdf_resp\
    .withColumn('email_cnt_dist', 
                    f.approx_count_distinct(f.col('email'))\
                    .over(Window.partitionBy('inn', 'kpp', 'campaign_nm', 'email')).cast('integer'))\
    .filter("email_cnt_dist > 1").show()

,inn,kpp,campaign_nm,email,email_proba,email_class,asdayname,ashour,day_open_cnt,fresh_user,load_dt,email_cnt_dist


In [159]:
hive.sql("select * from sbx_team_digitcamp.RESP_INN_KPP_CAMP_EMAIL_PROBA").count()

7108359

In [110]:
alter_spdf = \
hive.sql(
'''
select 
 inn
,kpp
,campaign_nm
,email
,email_proba
,email_class
,asdayname
,ashour
,day_open_cnt
,fresh_user
,load_dt
from(
        select 
            t.*,
            row_number() over (partition by inn, 
                                            kpp,
                                            campaign_nm,
                                            email, 
                                            email_proba 
                                                         order by load_dt) rownum
        from(
                select
                    coalesce(p.inn, b.inn_) inn,
                    coalesce(p.kpp, b.kpp_) kpp,
                    coalesce(p.campaign_nm, b.campaign_nm_) campaign_nm,
                    coalesce(p.email, b.email_) email,
                    coalesce(b.email_proba_, p.email_proba) email_proba,
                    coalesce(b.email_class_, p.email_class) email_class,
                    coalesce(b.asdayname_, p.asdayname) asdayname,
                    coalesce(b.ashour_, p.ashour) ashour,
                    coalesce(b.day_open_cnt_, p.day_open_cnt) day_open_cnt,
                    coalesce(b.fresh_user_, p.fresh_user) fresh_user,
                    coalesce(b.load_dt_, p.load_dt) load_dt
                from sbx_team_digitcamp.RESP_INN_KPP_CAMP_EMAIL_PROBA p
                full outer join tmp_scores b on
                p.inn = b.inn_ and p.kpp = b.kpp_ and
                p.email = b.email_
                
        ) t
)
where rownum = 1
'''
)

In [115]:
alter_spdf.count()

14984227

In [130]:
alter_spdf.select(f.date_format('load_dt', 'yyyy-MM-dd').alias('trunc_dt'),'inn').groupBy('trunc_dt').count().show()

,trunc_dt,count
0,2020-02-09,5494845
1,2020-07-26,10861431


In [120]:
table_name = 'RESP_INN_KPP_CAMP_EMAIL_PROBA'
tmp_tbl = 'tmp_{}'.format(table_name)

alter_spdf.registerTempTable(tmp_tbl)

hive.sql('''drop table if exists sbx_team_digitcamp.{}__'''.format(table_name))
hive.sql('''create table sbx_team_digitcamp.{}__ select * from {}'''.format(table_name,tmp_tbl))
hive.sql('''drop table if exists sbx_team_digitcamp.{}'''.format(table_name))
subprocess.call(['hdfs', 'dfs', '-rm', '-R', '-skipTrash', "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}".format(table_name.lower())])
hive.sql('''alter table sbx_team_digitcamp.{}__ rename to sbx_team_digitcamp.{}'''.format(table_name, table_name))
hive.sql("drop table if exists {}".format(tmp_tbl))

DataFrame[]